## 1. Khởi tạo & import

In [1]:
# thêm project root (thư mục chứa 'src/') vào sys.path
from pathlib import Path
import sys

ROOT = Path.cwd()
while not (ROOT / "src").is_dir() and ROOT.parent != ROOT:
    ROOT = ROOT.parent

sys.path.append(str(ROOT))

In [ ]:
import numpy as np
from src.data_processing import build_features, ensure_data_dirs, load_airbnb_numpy

## 2. Đọc dữ liệu (CSV => NumPy)

## 3. Hàm helpers thuần NumPy 

## 4. Làm sạch dữ liệu

## 5. Tạo đặc trưng 

## 6. Lưu output .npz để dùng cho 03_modeling

## 7) Sanity check 

In [ ]:
from src import preprocess_and_save
out_path = preprocess_and_save(root="../data", out_name="airbnb_2019_preprocessed.npz")
print("Saved ->", out_path)


In [ ]:
import numpy as np

bundle = np.load(out_path, allow_pickle=True)
X, y = bundle["X"], bundle["y"]
assert np.isfinite(X).all(), "X co NaN/Inf"
assert np.isfinite(y).all(), "y co NaN/Inf"
print("Shapes:", X.shape, y.shape)


In [ ]:
dirs = ensure_data_dirs(root=str(ROOT / "data"))
csv_path = dirs["raw"] / "AB_NYC_2019.csv"
data = load_airbnb_numpy(csv_path)
bundle = build_features(data["num"], data["text"])

out_path = dirs["processed"] / "airbnb_2019_preprocessed.npz"
np.savez_compressed(
    out_path,
    X=bundle.X,
    y=bundle.y,
    y_log1p=bundle.y_log1p,
    feature_names=bundle.feature_names,
    room_type=bundle.room_type_uniques,
    neighbourhood_group=bundle.neigh_group_uniques,
)
print("Saved ->", out_path)

In [ ]:
import numpy as np

bundle = np.load(out_path, allow_pickle=True)
X = bundle["X"]
y = bundle["y"]

print("Shapes:", X.shape, y.shape)
print("Feature count:", bundle["feature_names"].shape[0])
print("room_type labels:", bundle["room_type"].tolist())
print("neighbourhood_group labels:", bundle["neighbourhood_group"].tolist())
print("NaN in X:", int(np.isnan(X).sum()), "| NaN in y:", int(np.isnan(y).sum()))

assert np.isfinite(X).all(), "X contains NaN/Inf"
assert np.isfinite(y).all(), "y contains NaN/Inf"